# The ScikitLearn.jl library

The Scikit-learn library is an open-source machine learning library developed for the Python programming language, the first version of which dates back to 2010. It implements many machine learning models, related to classification, regression, clustering or dimensionality reduction. These models include Support Vector Machines (SVM), decision trees, random forests, or k-means. It is currently one of the most widely used libraries in the field of machine learning, due to the large number of functionalities it offers as well as its ease of use, since it provides a uniform interface for training and using models. The documentation for this library is available at https://scikit-learn.org/stable/.

For Julia, the ScikitLearn.jl library implements this interface and the algorithms contained in the scikit-learn library, supporting both Julia's own models and those of the scikit-learn library. The latter is done by means of the PyCall.jl library, which allows code written in Python to be executed from Julia in a transparent way for the user, who only needs to have ScikitLearn.jl installed. Documentation for this library can be found at https://scikitlearnjl.readthedocs.io/en/latest/.

However, recently, some incompatibilities have been reported with some versions of the SSL library. To avoid potential compatibility issues between Julia, PyCall, and ScikitLearn, we will use a different library for this exercise.
The library we will use is MLJ (Machine Learning in Julia), which is not strictly a library but rather a framework that allows the use of various related libraries through a common interface.
As a result, the function names used to create and train models remain the same regardless of the specific models being used.
In the practical sessions of this course, in addition to ANNs, we will use the following models, available within the MLJ framework:

- Support Vector Machines (SVM)
- Decision trees
- kNN

In order to use these models, it is first necessary to install and import the library:

In [19]:
import Pkg;
using MLJ;

Similarly, it is necessary to install the packages that contain the specific learning algorithms (e.g., LIBSVM, NearestNeighborModels, DecisionTree) as well as the packages that provide the interfaces between these algorithms and the MLJ framework (MLJLIBSVMInterface, MLJDecisionTreeInterface).
To import the models to be used, we can rely on the `MLJ.@load` macro. For example, the following lines import the three models mentioned above, which will be used in this course:

In [20]:
Pkg.add("MLJ")
Pkg.add("MLJBase")
Pkg.add("MLJLIBSVMInterface");
Pkg.add("NearestNeighborModels");
Pkg.add("MLJDecisionTreeInterface");
Pkg.add("CategoricalArrays")
Pkg.add("LIBSVM"); using LIBSVM;
using Random;


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.to

In [21]:
SVMClassifier = MLJ.@load SVC pkg=LIBSVM verbosity=0
kNNClassifier = MLJ.@load KNNClassifier pkg=NearestNeighborModels verbosity=0
DTClassifier = MLJ.@load DecisionTreeClassifier pkg=DecisionTree verbosity=0

MLJDecisionTreeInterface.DecisionTreeClassifier

As can be seen, each model is loaded from a different package. The `verbosity=0` option is simply used to suppress the output message that would otherwise be printed during the import.
This way, we define three functions to create each one of the three models. Each function receives as arguments the specific hyperparameters for the corresponding model.
Below are three examples, one for each type of model that will be used in these course exercises:

In [22]:
model = SVMClassifier(kernel=LIBSVM.Kernel.RadialBasis, cost=1.0, gamma=2.0, degree=Int32(3))

SVC(
  kernel = LIBSVM.Kernel.RadialBasis, 
  gamma = 2.0, 
  cost = 1.0, 
  cachesize = 200.0, 
  degree = 3, 
  coef0 = 0.0, 
  tolerance = 0.001, 
  shrinking = true)

In [23]:
model = DTClassifier(max_depth=4, rng=Random.MersenneTwister(1))

DecisionTreeClassifier(
  max_depth = 4, 
  min_samples_leaf = 1, 
  min_samples_split = 2, 
  min_purity_increase = 0.0, 
  n_subfeatures = 0, 
  post_prune = false, 
  merge_purity_threshold = 1.0, 
  display_depth = 5, 
  feature_importance = :impurity, 
  rng = MersenneTwister(1))

In [24]:
model = kNNClassifier(K=3)

KNNClassifier(
  K = 3, 
  algorithm = :kdtree, 
  metric = Distances.Euclidean(0.0), 
  leafsize = 10, 
  reorder = true, 
  weights = NearestNeighborModels.Uniform())

When creating a kNN model, the main hyperparameter is `K`, which defines the number of neighbors.
For decision trees, the main hyperparameter is `max_depth`, which sets the maximum depth of the tree.
In the case of decision trees, as shown earlier, there is also a parameter called `rng`. This parameter controls the randomness involved in a specific part of the model construction process.

Specifically, for decision trees, this randomness occurs during the selection of features used to split a node. The `DecisionTree` library uses a random number generator (RNG) for this step, which is updated with each call. As a result, different calls to the function (along with subsequent calls to `fit!`) may produce different models, even with the same data.

To control this randomness and make the process deterministic, it is advisable to provide a fixed integer value as the RNG seed, as shown in the previous example.
This ensures that creating a model with a given input-output dataset and a defined set of hyperparameters becomes a reproducible process.

In general, it is preferable to control randomness across the entire model development workflow (e.g., cross-validation, training/test splits) by setting a global random seed at the beginning.
However, for the purposes of these exercises, we will use the `rng` keyword specifically for the decision tree model.

SVMs have a more complex set of hyperparameters, which depend on the kernel function being used.  
First, the hyperparameter `C` controls the trade-off between the margin width and classification error. Lower values allow for more misclassifications (more tolerance), while higher values fit the model more tightly to the data.  
In MLJ, this parameter is passed using the keyword `cost` when calling `SVMClassifier`.

Additionally, it is necessary to specify which kernel to use. This is done using the keyword `kernel`, which can take one of the following values provided by the `LIBSVM` library:

- `LIBSVM.Kernel.Linear`
- `LIBSVM.Kernel.RadialBasis`
- `LIBSVM.Kernel.Sigmoid`
- `LIBSVM.Kernel.Polynomial`

Depending on the kernel selected, different additional hyperparameters are used:

- **Linear kernel**: Only requires `C` (via `cost`).
- **RBF (Radial Basis Function) kernel**: In addition to `C`, it uses `gamma`, which controls the influence of each support vector.
- **Sigmoid kernel**: Uses `C`, `gamma`, and `coef0`. This kernel behaves similarly to a neural network, where `gamma` and `coef0` influence the shape of the decision function.
- **Polynomial kernel**: Uses `C`, `degree` (the degree of the polynomial), `gamma`, and `coef0`.

Typical values for these hyperparameters include:  
`0.001`, `0.1`, `1`, `10`, `100`, `1000`.

The following table summarises the different hyperparameters, the kernels that use them, and typical values they may take.

Note that when calling the `SVMClassifier` function, the hyperparameters use the same names as listed here, except for `C`, which must be passed as `cost`, as shown in the previous example.

It is also important that the arguments passed to the function have the correct type, as required by the `LIBSVM` library. Otherwise, an error may occur.

To prevent this, it is recommended to explicitly cast each hyperparameter to the appropriate type when calling the function.

| Hyperparameter | Applicable Kernels                 | Typical Values                     | Required Type in LIBSVM |
|----------------|------------------------------------|------------------------------------|--------------------------|
| `cost` (`C`)   | Linear, RBF, Sigmoid, Polynomial   | 0.001, 0.1, 1, 10, 100, 1000       | `Float64`               |
| `gamma`        | RBF, Sigmoid, Polynomial           | 0.1, 0.01, 0.001, 0.0001           | `Float64`               |
| `coef0`        | Sigmoid, Polynomial                | 0, 1, 5, 10                        | `Int32`                 |
| `degree`       | Polynomial                         | 2, 3, 4, 5                         | `Float64`               |

Although the basic SVM model is inherently binary, the implementation provided in MLJ already supports multi-class classification.  
Therefore, it is not necessary to manually apply a one-vs-all strategy for multi-class problems.

Once a model has been created, it must be wrapped in a `machine` object. This object acts as a container that associates the model with the data and handles both training and prediction.  
It is a core concept in MLJ and simplifies model workflows by centralizing model fitting (`fit!`) and prediction (`predict`) logic.

A `machine` has three main components:

- **Model**: Specifies the algorithm to be used. It has been created earlier, without any data or learned state.
- **Data**: Provides the input features and target labels (if supervised).
- **Internal State**: Stores learned parameters after the model is trained.

To create a `machine`, you can use the `machine` function, passing in the model, the input features, and the target labels.  
Note that the input data must not be plain arrays. Instead, it should be converted to a supported table format such as `Tables.table`, `DataFrame`, or a `NamedTuple`.  
If your data is currently stored as arrays, as is the case in these exercises, the following line shows how to construct the machine:

In [25]:
using DelimitedFiles
using Flux
include("utils.jl")
using .Utils

In [26]:
using DelimitedFiles
using Random
using CategoricalArrays

# -----------------------------
# Configuration
# -----------------------------
topology      = [4, 3]     # Hidden layers
learningRate  = 0.01
numMaxEpochs  = 1000
Pval          = 0.2        # Validation split proportion
Ptest         = 0.2        # Test split proportion
rng           = MersenneTwister(1234) 

# -----------------------------
# Load and parse the dataset
# -----------------------------
dataset = readdlm("./iris/iris.data", ',')

# Features (N×4) as Float32
inputs = convert(Array{Float32,2}, dataset[:, 1:4])

# Original string labels (N)
labels = Vector{String}(dataset[:, 5])

# Class list (kept for one-hot encoding with Utils)
classes = unique(labels)

# -----------------------------
# One-hot targets (N×C, Bool) via Utils
# -----------------------------
targets_oh = Utils.oneHotEncoding(labels, classes)  # shape: N × C (Bool)

# -----------------------------
# Train/Validation/Test split (by indices) using Utils.holdOut
# -----------------------------
N = size(inputs, 1)
train_idx, val_idx, test_idx = Utils.holdOut(N, Pval, Ptest)

# Subset feature matrices
train_inputs_raw = inputs[train_idx, :]
val_inputs_raw   = inputs[val_idx,   :]
test_inputs_raw  = inputs[test_idx,  :]

# Subset one-hot targets (optional, for your ANN with Utils)
train_targets_oh = targets_oh[train_idx, :]  # shape: N_train × C
val_targets_oh   = targets_oh[val_idx,   :]  # shape: N_val × C
test_targets_oh  = targets_oh[test_idx,  :]  # shape: N_test × C

# -----------------------------
# Targets as a categorical vector (required by MLJ/LIBSVM)
#   - MLJ’s supervised models expect a 1D categorical target, not one-hot.
# -----------------------------
y_all         = categorical(labels)      # CategoricalVector{String}
train_targets = y_all[train_idx]         # <- this is what Code B will use
val_targets   = y_all[val_idx]
test_targets  = y_all[test_idx]

println("Sizes -> train: $(size(train_inputs_raw,1)), val: $(size(val_inputs_raw,1)), test: $(size(test_inputs_raw,1))")

# -----------------------------
# Min–Max normalization with Utils
# -----------------------------
norm_params = Utils.calculateMinMaxNormalizationParameters(train_inputs_raw)

train_inputs = Float32.(Utils.normalizeMinMax(train_inputs_raw, norm_params))  # <- used by Code B
val_inputs   = Float32.(Utils.normalizeMinMax(val_inputs_raw,   norm_params))
test_inputs  = Float32.(Utils.normalizeMinMax(test_inputs_raw,  norm_params))

trainDS = (train_inputs, train_targets_oh)
valDS   = (val_inputs,   val_targets_oh)
testDS  = (test_inputs,  test_targets_oh)

Sizes -> train: 90, val: 30, test: 30


(Float32[0.71428573 0.5333333 0.7457627 0.83333325; 0.5142856 -0.0666666 0.59322035 0.5833333; … ; 0.17142855 0.4666667 0.10169492 0.041666664; 0.19999994 0.8 0.0677966 0.083333336], Bool[0 0 1; 0 1 0; … ; 1 0 0; 1 0 0])

In [27]:
# define the model
model = SVMClassifier(kernel=LIBSVM.Kernel.RadialBasis, cost=1.0, gamma=2.0, degree=Int32(3))
# create the machine object
mach = machine(model, MLJ.table(train_inputs), categorical(train_targets))

untrained Machine; caches model-specific representations of data
  model: SVC(kernel = RadialBasis, …)
  args: 
    1:	Source @318 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @146 ⏎ AbstractVector{Multiclass{3}}


As shown, the input matrix is converted into a table, and the target vector is converted into a categorical array, since this exercise involves classification problems.  

It is important to note that the variable `targets` (the output labels) should be a **vector**, not a matrix. Each element in the vector corresponds to the label of one input sample and can be of any type (e.g., integer, string, etc.).

Although some models may support one-hot encoded labels, others do not. Therefore, in these exercises, we will use a vector of labels, with one label per instance, rather than one-hot encoding (which is typically used in neural networks).

To prevent compatibility issues with certain model implementations, we convert all label values to `String` before passing them to the model.

Once the machine object has been created, the model can be trained using the `fit!` function as follows:

In [28]:
MLJ.fit!(mach, verbosity=0)

trained Machine; caches model-specific representations of data
  model: SVC(kernel = RadialBasis, …)
  args: 
    1:	Source @318 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @146 ⏎ AbstractVector{Multiclass{3}}


This function only requires the `machine` object as an argument, since the training data has already been bound to it.
The optional argument `verbosity=0` is used to suppress output messages during training.

### Question 6.1

> ❓ What does the fact that the name of this function ends in bang (!) indicate?

In Julia, the exclamation mark (`!`) at the end of a function’s name is a naming convention that indicates the function modifies its argument(s) in place or has side effects.

Therefore, in the case of `fit!(mach)`, the ! denotes that this function changes the internal state of the `machine` object — specifically, it performs the training process and stores the learned model parameters within the object.

Contrary to the Flux library, where it was necessary to write the ANN training loop, in this library the loop is already implemented, and it is called automatically when the `fit!` function is executed. Therefore, it is not necessary to write the code for the training loop.

An important aspect to consider is the layout of the data to be used.  

As shown in previous exercises, when training an Artificial Neural Network (ANN), the input samples (patterns) are arranged in **columns**, and each **row** in the input matrix represents a feature.

However, outside the scope of ANNs — and therefore for all other techniques used in this course — it is assumed that the samples are arranged in **rows**, meaning each **column** in the input matrix corresponds to a feature. This format is generally more intuitive and will be used throughout the rest of the course.


### Question 6.2

> ❓ As in the case of ANNs, a loop is necessary for training several models. Where in the code (inside or outside the loop) will you need to create the model? Which models will need to be trained several times and which ones only once? Why?

The model should be **created outside the loop** as a reusable template that defines the algorithm and its hyperparameters, while the **machine object should be instantiated and trained inside the loop** for each iteration (for example, for every cross-validation fold or random seed). This is because `fit!` modifies the machine in place, and each training cycle must start from an unfitted state to ensure independent and unbiased results.

Models that involve randomness or depend on data partitions — such as neural networks, stochastic algorithms, or models under cross-validation or hyperparameter tuning — must be trained **multiple times**, once for each iteration or configuration. In contrast, **deterministic models** with fixed hyperparameters (for example, logistic regression or SVMs) need to be trained **only once** on the full training data after model selection.


### Question 6.3

> ❓ Which condition must the matrix of inputs and the vector of desired outputs passed as an argument to this function fulfil?

The matrix of inputs (`X`) and the vector of desired outputs (`y`) must have **the same number of samples**. In other words, the **number of rows in the input matrix must match the length of the output vector**, since each row in `X` represents the set of features corresponding to a single training instance, and each element in `y` provides the target label for that same instance.

Additionally, the data types must be consistent with the model’s requirements: the inputs should contain **continuous numerical values** (for example, `Float32` or `Float64`), and the output vector should be **categorical** in classification problems or **numeric** in regression problems.


Finally, once the model has been trained, it can be used to make predictions. This is done using the `predict` function. The following is an example of how to use it:

In [29]:
testOutputs = MLJ.predict(mach, MLJ.table(test_inputs));

As shown, the `predict` function requires two arguments: the `machine` object and the input matrix, which must be converted to a table format.

In classification problems, the type of the prediction result depends on the model and the underlying library:

- **For SVMs**, `predict` returns a `CategoricalArray`, which can be directly compared with the ground truth labels. No post-processing is needed.
- **For Decision Trees and kNN**, `predict` returns a `UnivariateFiniteArray`, which represents a probability distribution over the possible classes.  
  To convert this into a single predicted label (so it can be compared with the true values), you can use the `mode` function to extract the most likely class.

The model being used is stored in memory as a structured object with several fields, and it can be very useful to inspect its contents.  
The `machine` object holds the model, the data, and the results of training. Therefore, you can access the trained model through the `machine`, or more directly through the variable `model`.

For example, when training an SVM, you can access one of its hyperparameters in either of the following ways:

In [30]:
model.gamma
mach.model.gamma

2.0

To inspect the learned parameters after training, MLJ provides several options.

One particularly interesting case is with SVMs, where it is useful to check which instances were selected as support vectors.  
This can be done in two ways:

In [31]:
mach.fitresult[1].SVs.indices

39-element Vector{Int32}:
  1
  3
  6
  8
 12
 15
 18
 26
 32
 33
  ⋮
 61
 70
 74
 80
 20
 31
 35
 37
 51

or using the higher-level MLJ interface:

In [32]:
fitted_params(mach)[:libsvm_model].SVs.indices

39-element Vector{Int32}:
  1
  3
  6
  8
 12
 15
 18
 26
 32
 33
  ⋮
 61
 70
 74
 80
 20
 31
 35
 37
 51

These commands return the indices of the support vectors in the training dataset.

In this notebook, the task will be to develop a single function that allows training the three different models using the MLJ library, and, in addition, artificial neural networks (ANNs) using the functions developed in previous exercises.

The training will be performed using cross-validation. For each fold, the specified model will be trained, and metrics will be computed on the test set.

As in the previous exercise, it is useful to generate a confusion matrix that reflects the distribution of instances across the test sets. In this case, it is simpler than before because the methods used are deterministic, so only one confusion matrix will be created per fold, and the final confusion matrix will be the sum of all of them.

Nevertheless, the considerations from the previous exercise still apply — in particular, that the metrics derived from this global confusion matrix may not match the metrics obtained through cross-validation.

In this exercise, you will develop a single function called `modelCrossValidation` that, in addition to training artificial neural networks (ANNs), performs cross-validation for SVMs, decision trees, and kNN.

The function should receive the following arguments:

- **`modelType::Symbol`**: This parameter indicates the type of model to train. It should take one of the following values:
  - `:ANN` — Artificial Neural Network
  - `:SVC` — Support Vector Machine
  - `:DecisionTreeClassifier` — Decision Trees
  - `:KNeighborsClassifier` — k-Nearest Neighbors

- **`modelHyperparameters::Dict`**: A dictionary containing the model's hyperparameters. Keys may be of type `String` or `Symbol`.
  
  To check whether a hyperparameter is defined, you can use `haskey`.  
  To retrieve a value that may or may not exist in the dictionary, the `get` function is also useful.

  - **ANN (`:ANN`)**:
        The expected Hyperparameters are
        - Topology (number of hidden layers and number of neurons in each hidden layer, required) and transfer funtion in each layer. In "shallow" networks such as those used in this course, the transfer function has less impact, so a standard one, shuch as `tansig` or `logsig`, can be used.
        - Learning rate
        - Ratio of patterns used for validation
        - Number of consecutive iterations without improving the validation loss to stop the process
        - Number of times each ANN is trained.
        
### Question 6.4    
> ❓ Why should a linear transfer function not be used for neurons in the hidden layers?

A **linear transfer function** should not be used for neurons in the hidden layers because it does **not introduce nonlinearity** into the model. Without nonlinear activation functions, the entire network, regardless of how many hidden layers it has, would behave as a **single linear transformation** of its inputs. In other words, stacking multiple linear layers simply results in another linear function, providing no additional representational power.

The purpose of hidden layers is to allow the network to learn **complex, nonlinear relationships** between the input features and the output targets. Nonlinear activation functions (such as sigmoid, tanh, or ReLU) enable the model to approximate arbitrary nonlinear mappings, which are essential for solving complex classification or regression tasks that cannot be captured by a purely linear model.

  For the other models, the expected hyperparameters are:

  - **SVM (`:SVC`)**:  
    The expected hyperparameters are:
    - `C`
    - `kernel`
    - `degree`
    - `gamma`
    - `coef0`

    The `kernel` parameter should be provided as a `String` with one of the following values:
  `"linear"`, `"rbf"`, `"sigmoid"`, or `"poly"`.

    Depending on the selected kernel, some of the hyperparameters may be ignored. For example:
    - The `"poly"` kernel uses `degree`, `gamma`, and `coef0`.
    - The `"sigmoid"` kernel uses `gamma` and `coef0`.
    - The `"linear"` kernel only uses `C`.

    The `C` hyperparameter must be passed using the keyword `cost`, and the kernel must be translated to one of the predefined constants in the `LIBSVM` library:

    - `LIBSVM.Kernel.Linear`
    - `LIBSVM.Kernel.RadialBasis`
    - `LIBSVM.Kernel.Sigmoid`
    - `LIBSVM.Kernel.Polynomial`

    To avoid type errors, it is recommended to cast each value explicitly.  
    For example, to create a polynomial SVM:

  ```julia
    model = SVMClassifier(
        kernel = LIBSVM.Kernel.Polynomial,
        cost = Float64(C),
        gamma = Float64(gamma),
        degree = Int32(degree),
        coef0 = Float64(coef0)
    )
  ```

  - **Decision Tree (`:DecisionTreeClassifier`)**:

    - `max_depth`: defines the maximum depth of the tree.
    - `rng`: the random seed generator. It should be set to `Random.MersenneTwister(1)` to ensure reproducibility.

  - **k-Nearest Neighbors (`:KNeighborsClassifier`)**:
    - `n_neighbors`: the value of k, which determines the number of neighbors to consider.

- **`dataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{<:Any,1}}`**:  
  A tuple containing two elements:
  - The first is the input matrix (`X`). Unlike neural network training, there is no need to convert the data to `Float32`, since both `Float32` and `Float64` are commonly used in this library depending on the desired precision.
  - The second is the target vector (`y`), which contains the labels.

- **`crossValidationIndices::Array{Int64,1}`**:  
  This vector contains the indices used to assign each sample to a fold in the cross-validation process.

  As in the previous exercise, the fold assignment must be done **outside** the `modelCrossValidation` function.  
  This ensures that the exact same data partitioning is used when training different models, allowing fair comparisons.


The function will begin by checking whether the model to be trained is a neural network, by examining the `modelType` parameter.  
If this is the case, it will call the `ANNCrossValidation` function, passing the hyperparameters provided in `modelHyperparameters`.

Keep in mind that many of the hyperparameters for neural networks may not be defined in the dictionary.  
As mentioned earlier, the function `haskey` can be used to check whether a key is present in a `Dict`.  
Alternatively, the `get` function can be used to safely retrieve a value with a default if the key is missing.

Once the call to `ANNCrossValidation` is made, the function returns its result and exits — meaning that no further processing will occur in this case.

If a different type of model is to be trained, the logic continues similarly to the previous exercise:

- Create seven vectors to store the results of the metrics for each fold.
- Create a 2D array to accumulate the confusion matrix, initialized with zeros.

A key modification when using models from the MLJ library is to **convert the target labels to strings** before training any model.  
This helps prevent errors caused by internal type mismatches in some model implementations.

This can be done with the following simple line:

```julia
targets = string.(targets);
```

Additionally, it will be necessary to compute the vector of unique classes, just like in the previous exercise.  
This can be done with:

```julia
classes = unique(targets);
```

Once these initial steps are completed, the cross-validation loop can begin.

In each iteration, the following steps are performed:

1. Extract the training and test input matrices and the corresponding target vectors.  
   These should be of type `AbstractArray{<:Any,1}` for the targets.

2. Create the model with the specified hyperparameters.

3. For MLJ models (SVM, Decision Tree, kNN):
   - Instantiate the model using the appropriate constructor: `SVMClassifier`, `DTClassifier`, or `kNNClassifier`, depending on `modelType`.
   - Wrap the model in a `machine` with the training data.
   - Train the model using `fit!`.

4. Perform predictions on the test data using `predict`.

   - For Decision Trees and kNN, use `mode` to convert the probabilistic predictions into categorical labels:
     ```julia
     ŷ = mode.(predict(mach, MLJ.table(Xtest)))
     ```

   - For SVMs, the output of `predict` can be compared directly with the ground truth, since it returns a `CategoricalArray`.

Although the general structure of the code will be the same for the three model types, each model requires a different constructor and may require post-processing (e.g., `mode`) depending on the prediction format.

Once the predicted labels for the test set are available, the evaluation metrics and the confusion matrix should be computed using the `confusionMatrix` function.

- The metrics returned should be stored in their respective positions within the metric vectors.
- The confusion matrix obtained for each fold should be **added** to a global confusion matrix for the test set.

A key difference compared to the ANN training in the previous exercise is that these models (SVM, Decision Tree, kNN) are **deterministic**.  
Therefore, each model only needs to be trained **once per fold**, without requiring multiple executions or averaging across runs.

   ### Question 6.5
   > ❓ The other models do not have the number of times to train them as a parameter. Why? If you train several times, Which statistical properties will the results of these trainings have?

Unlike ANNs, the other models such as SVMs, Decision Trees, and k-Nearest Neighbors are **deterministic** algorithms. This means that, given the same training data and hyperparameters, they will always produce the same model and the same predictions, there is no randomness involved in their optimization or initialization process. Consequently, there is no need to repeat their training multiple times, as doing so would yield identical results.

If these models were trained multiple times under identical conditions, the results of their metrics would exhibit **zero variance**, since the outcomes would be perfectly reproducible. In contrast, repeating the training of stochastic models like ANNs allows one to estimate the **statistical variability** of their performance, providing a measure of the model’s stability and sensitivity to random initialization.

As previously described, when using techniques such as SVM, decision trees, kNN, or DoME, **one-hot encoding is not used**.  
Instead, metrics are computed using the `confusionMatrix` function developed in a previous exercise, which takes three arguments:
- The predicted labels
- The true labels
- The list of class labels

All of these must be of type `AbstractArray{<:Any,1}`.

It is important to use the version of the `confusionMatrix` function that receives the vector of classes.

### Question 6.6

> ❓ What could happen if the version that does not receive the class vector is used?

If the version of the `confusionMatrix` function that does **not** receive the explicit class vector is used, the function will automatically infer the set of classes by combining the labels found in the predicted and true output vectors. This can lead to **inconsistent or incorrect class ordering**, especially when some classes are missing in the predictions or in a particular test fold.

As a result, the confusion matrix may have **misaligned rows and columns**, meaning that the counts of true and predicted labels would not correspond to the same class positions. This would produce incorrect metric calculations (such as accuracy, precision, or recall) and potentially misleading interpretations of model performance. Providing the explicit list of classes ensures that the confusion matrix maintains a consistent and correct class ordering across all folds and models.

The `modelCrossValidation` function must return the same structure as in the previous exercise: a tuple with 8 elements.

- The first **7 elements** correspond to metrics: **accuracy**, **error rate**, **recall**, **specificity**, **PPV**, **NPV**, and **F1-score**.  
  Each of these is itself a tuple with the **mean** and **standard deviation** across the folds.

- The **8th element** is the **global confusion matrix** computed on the test sets.

Once the function has been developed, it can be used to evaluate different model configurations by comparing test results across the selected metrics.  
This process does **not return a final model ready for production**, but rather identifies the best-performing model type and hyperparameter configuration.

After selecting the best configuration, the final model should be trained **from scratch** using **all available data**, without performing cross-validation.  
This training is done just once, without setting aside a test set.  
As a result, the final model is expected to perform slightly better than during cross-validation, since it benefits from more training data.

This final model is the one intended for production use, and a confusion matrix can be computed for it as well.

### Question 6.7

> ❓ In the case of using decision trees or kNN, a corresponding function is not necessary to perform the "one-against-all" strategy, why?

In the case of Decision Trees and k-Nearest Neighbors (kNN), a separate “one-against-all” strategy is unnecessary because these algorithms are **inherently multiclass classifiers**. Both methods are designed to handle multiple classes natively, Decision Trees split the data recursively based on feature values to directly assign samples to one of several possible classes, while kNN determines the class of a sample by **majority voting** among its *k* nearest neighbors, which can belong to any of the available classes.

In contrast, models such as Support Vector Machines (SVMs) were originally formulated for **binary classification**, separating only two classes with a single hyperplane. Extending SVMs to multiclass problems requires strategies like “one-against-all” or “one-against-one,” which combine multiple binary classifiers to make a final multiclass prediction. Decision Trees and kNN do not require such decomposition because they naturally operate with any number of class labels.

In [33]:
using Random, Statistics
using MLJBase
using MLJBase: UnivariateFinite
using CategoricalArrays
using LIBSVM 
using .Utils

# --- MLJ model wrappers (do NOT use raw LIBSVM.SVC directly) ---
const SVCModel = @load SVC pkg=LIBSVM verbosity=0
const DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree verbosity=0
const KNNModel = @load KNNClassifier pkg=NearestNeighborModels verbosity=0

# --- Always return hard labels as Vector{String}, regardless of model output ---
function safe_predict_labels(mach, Xnew)::Vector{String}
    yhat = MLJ.predict(mach, MLJ.table(Xnew))
    if yhat isa AbstractVector{<:UnivariateFinite}
        return String.(mode.(yhat))
    elseif yhat isa AbstractVector
        return String.(yhat)
    else
        error("Unexpected prediction type: $(typeof(yhat))")
    end
end

safe_predict_labels (generic function with 1 method)

In [34]:
function modelCrossValidation(
    modelType::Symbol, modelHyperparameters::Dict,
    dataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{<:Any,1}},
    crossValidationIndices::Vector{Int}
)

    # ---------------------------
    # Unpack and basic validation
    # ---------------------------
    X_raw, y_raw = dataset
    N = size(X_raw, 1)
    @assert length(y_raw) == N "Length of y must match number of rows in X."
    @assert length(crossValidationIndices) == N "CV index length must match N."
    numFolds = maximum(crossValidationIndices)
    @assert minimum(crossValidationIndices) ≥ 1 "Fold indices must start at 1."
    @assert all(in(1:numFolds), crossValidationIndices) "Fold indices must be in 1..numFolds."

    # Canonical types for MLJ backends
    X = Array{Float64}(X_raw)
    y_str = string.(y_raw)
    classes = unique(y_str)
    C = length(classes)

    # ----------------------------------------
    # ANN branch: delegate and return directly
    # ----------------------------------------
    if modelType == :ANN
        topology        = get(modelHyperparameters, :topology, nothing)
        @assert topology !== nothing "For :ANN you must provide :topology (hidden layer sizes)."

        transferFuncs   = get(modelHyperparameters, :transferFunctions, nothing)
        maxEpochs       = get(modelHyperparameters, :maxEpochs,       200)
        minLoss         = get(modelHyperparameters, :minLoss,         0.0)
        learningRate    = get(modelHyperparameters, :learningRate,    1e-3)
        validationRatio = get(modelHyperparameters, :validationRatio, 0.0)
        maxEpochsVal    = get(modelHyperparameters, :maxEpochsVal,    25)
        numExecutions   = get(modelHyperparameters, :numExecutions,   1)

        return ANNCrossValidation(
            topology,
            (X, y_str),
            crossValidationIndices;
            numExecutions     = numExecutions,
            transferFunctions = transferFuncs,
            maxEpochs         = maxEpochs,
            minLoss           = minLoss,
            learningRate      = learningRate,
            validationRatio   = validationRatio,
            maxEpochsVal      = maxEpochsVal
        )
    end

    # ----------------------------------------------------
    # Non-ANN models: accumulators (once per fold)
    # ----------------------------------------------------
    acc_f  = zeros(Float64, numFolds)
    err_f  = zeros(Float64, numFolds)
    sens_f = zeros(Float64, numFolds)
    spec_f = zeros(Float64, numFolds)
    ppv_f  = zeros(Float64, numFolds)
    npv_f  = zeros(Float64, numFolds)
    f1_f   = zeros(Float64, numFolds)
    globalCM = zeros(Float64, C, C)

    # ---------------------
    # Helpers: model build
    # ---------------------
    _kernel_of = function (kstr::AbstractString)
        ks = lowercase(strip(kstr))
        if     ks == "linear"   ; return LIBSVM.Kernel.Linear
        elseif ks == "rbf"      ; return LIBSVM.Kernel.RadialBasis
        elseif ks == "sigmoid"  ; return LIBSVM.Kernel.Sigmoid
        elseif ks == "poly" || ks == "polynomial"
                                 return LIBSVM.Kernel.Polynomial
        else
            error("Unknown SVM kernel: '$kstr'. Expected: linear | rbf | sigmoid | poly.")
        end
    end

    # --------------------------
    # Cross-validation main loop
    # --------------------------
    for fold in 1:numFolds
        test_mask  = crossValidationIndices .== fold
        train_mask = .!test_mask

        Xtr, Xte = X[train_mask, :], X[test_mask, :]
        ytr_str, yte_str = y_str[train_mask], y_str[test_mask]

        # 1) Build and fit the model for this fold (one fit per fold)
        if modelType == :SVC
            # Hyperparameters (with defaults)
            Cval   = Float64(get(modelHyperparameters, :C,      1.0))
            gamma  = Float64(get(modelHyperparameters, :gamma,  0.0))   # 0 => 1/n_features in LIBSVM
            degree = Int32(get(modelHyperparameters, :degree,   3))     # Int32 required by LIBSVM
            coef0  = Float64(get(modelHyperparameters, :coef0,  0.0))
            kern_s = String(get(modelHyperparameters, :kernel, "rbf"))
            kern   = _kernel_of(kern_s)

            model  = SVCModel(kernel = kern, cost = Cval, gamma = gamma, degree = degree, coef0 = coef0)
            mach   = machine(model, MLJ.table(Xtr), categorical(ytr_str))
            MLJ.fit!(mach; verbosity=0)

            yhat = safe_predict_labels(mach, Xte)

        elseif modelType == :DecisionTreeClassifier
            max_depth = get(modelHyperparameters, :max_depth, nothing)
            rng_dt    = get(modelHyperparameters, :rng, Random.MersenneTwister(1))

            model = DecisionTreeClassifier(max_depth = max_depth, rng = rng_dt)
            mach  = machine(model, MLJ.table(Xtr), categorical(ytr_str))
            MLJ.fit!(mach; verbosity=0)

            yhat = safe_predict_labels(mach, Xte)

        elseif modelType == :KNeighborsClassifier
            k = Int(get(modelHyperparameters, :n_neighbors, 5))

            model = KNNModel(K = k)
            mach  = machine(model, MLJ.table(Xtr), categorical(ytr_str))
            MLJ.fit!(mach; verbosity=0)

            yhat = safe_predict_labels(mach, Xte)

        else
            error("Unknown modelType: $(modelType). Expected :ANN | :SVC | :DecisionTreeClassifier | :KNeighborsClassifier")
        end

        # 2) Evaluate on the test subset using explicit class list (Utils)
        acc, err, sens, spec, ppv, npv, f1, CM = Utils.confusionMatrix(yhat, yte_str, classes)

        acc_f[fold]  = acc
        err_f[fold]  = err
        sens_f[fold] = sens
        spec_f[fold] = spec
        ppv_f[fold]  = ppv
        npv_f[fold]  = npv
        f1_f[fold]   = f1

        globalCM .+= Float64.(CM)
    end

    # 3) Aggregate across folds (mean ± std) and return
    return (
        (mean(acc_f),  std(acc_f)),
        (mean(err_f),  std(err_f)),
        (mean(sens_f), std(sens_f)),
        (mean(spec_f), std(spec_f)),
        (mean(ppv_f),  std(ppv_f)),
        (mean(npv_f),  std(npv_f)),
        (mean(f1_f),   std(f1_f)),
        globalCM
    )
end


modelCrossValidation (generic function with 1 method)

In [35]:
using DelimitedFiles

# 1) Load the dataset (CSV with 5 columns: 4 numeric + 1 label)
dataset = readdlm("iris/iris.data", ',')

# 2) Separate features (X) and class labels (targets)
X = Float64.(dataset[:, 1:4])            # 150×4 matrix of features
targets = Vector{String}(dataset[:, 5])  # 150 labels (Setosa, Versicolor, Virginica)

# 3) Normalize each feature column to [0, 1] using Utils
Xn = Utils.normalizeMinMax(X)
@assert all(minimum(Xn, dims=1) .≈ 0.0)
@assert all(maximum(Xn, dims=1) .≈ 1.0)

# 4) Generate stratified fold indices for 5-fold cross-validation using Utils
k = 5
rng = Random.MersenneTwister(123)  # reproducibility for the fold assignment
cv_idx = Utils.stratified_kfold_indices(targets, k; rng=rng)

# 5) Define model type and hyperparameters, then run cross-validation
modelType = :SVC
modelHyperparameters = Dict(
    :C       => 1.0,
    :kernel  => "rbf",    # "linear" | "rbf" | "sigmoid" | "poly"
    :degree  => 3,        # cast to Int32 inside the function
    :gamma   => 0.25,     # ~ 1 / n_features
    :coef0   => 0.0
)

result = modelCrossValidation(
    modelType,
    modelHyperparameters,
    (Xn, targets),
    cv_idx
)

# 6) Unpack results and display metrics
((meanAcc,  stdAcc),
 (meanErr,  stdErr),
 (meanSens, stdSens),
 (meanSpec, stdSpec),
 (meanPPV,  stdPPV),
 (meanNPV,  stdNPV),
 (meanF1,   stdF1),
 globalCM) = result

println("=== K-Fold Cross-Validation on Iris Dataset (k = $k) — $(modelType) ===")
println("Accuracy:   $(round(meanAcc, digits=4)) ± $(round(stdAcc, digits=4))")
println("Error:      $(round(meanErr, digits=4)) ± $(round(stdErr, digits=4))")
println("Sensitivity:$(round(meanSens, digits=4)) ± $(round(stdSens, digits=4))")
println("Specificity:$(round(meanSpec, digits=4)) ± $(round(stdSpec, digits=4))")
println("PPV:        $(round(meanPPV, digits=4)) ± $(round(stdPPV, digits=4))")
println("NPV:        $(round(meanNPV, digits=4)) ± $(round(stdNPV, digits=4))")
println("F1-Score:   $(round(meanF1, digits=4)) ± $(round(stdF1, digits=4))")

println("\nGlobal Confusion Matrix (sum over folds):")
show(stdout, "text/plain", round.(globalCM, digits=2))
println()


=== K-Fold Cross-Validation on Iris Dataset (k = 5) — SVC ===
Accuracy:   0.96 ± 0.0149
Error:      0.04 ± 0.0149
Sensitivity:0.96 ± 0.0149
Specificity:0.98 ± 0.0075
PPV:        0.9646 ± 0.0113
NPV:        0.9812 ± 0.0065
F1-Score:   0.9598 ± 0.0152

Global Confusion Matrix (sum over folds):
3×3 Matrix{Float64}:
 50.0   0.0   0.0
  0.0  49.0   1.0
  0.0   5.0  45.0


### Learn Julia

#### Symbols and Dictionaries in Julia
One Julia type that is important for this exercise is the `Symbol` type. An object of this type can be any symbol you want, simply by typing its name after a colon (":"). In this practice, you can use it to indicate which model you want to train, for example, in the `modelCrossValidation` function, symbols will be used to indicate which model to train:

```julia
:KNeighborsClassifier, :SVC, :DecisionTreeClassifier, :ANN
```

#### Passing Model-Specific Parameters
This function will also require model-specific parameters to be passed.
The recommended way to do this is to define a variable of type Dict, which works similarly to Python dictionaries.

For instance, to define the hyperparameters for an artificial neural network (ANN):

```julia
  modelHyperparameters = Dict(
      "topology" => [5, 3],
      "learningRate" => 0.01,
      "validationRatio" => 0.2,
      "numExecutions" => 50,
      "maxEpochs" => 1000,
      "maxEpochsVal" => 6
  )
```

Another way to define the same dictionary:

```julia
  modelHyperparameters = Dict()
  modelHyperparameters["topology"] = topology
  modelHyperparameters["learningRate"] = learningRate
  modelHyperparameters["validationRatio"] = validationRatio
  modelHyperparameters["numExecutions"] = numRepetitionsANNTraining
  modelHyperparameters["maxEpochs"] = numMaxEpochs
  modelHyperparameters["maxEpochsVal"] = maxEpochsVal
```
To access a value, simply use:
```julia
  modelHyperparameters["topology"]
```
#### Example for SVM Parameters
You can also define hyperparameters for other models similarly.
For example, for an SVM:
```julia
  modelHyperparameters = Dict("C" => 1, "kernel" => "rbf", "gamma" => 2)
```
Or using the alternative form:

```julia
  modelHyperparameters = Dict()
  modelHyperparameters["C"] = 1
  modelHyperparameters["kernel"] = "rbf"
  modelHyperparameters["gamma"] = 2
```
Other kernels may require different parameters, such as `degree` and `coef0`.

When building the SVM model inside the function, you might write:
```julia
  if modelHyperparameters["kernel"] == "rbf"
    model = SVMClassifier(
        kernel = LIBSVM.Kernel.RadialBasis,
        cost = Float64(modelHyperparameters["C"]),
        gamma = Float64(modelHyperparameters["gamma"])
    )
```

You can apply a similar strategy for decision trees, kNN, and DoME models.

In the examples above, the dictionary keys are `String`, but you may also use `Symbol` keys interchangeably.
For example:
```julia
  modelHyperparameters = Dict(:C => 1, :kernel => "rbf", :gamma => 2)
```

Another type of Julia that may be interesting for this assignment is the `Symbol` type. An object of this type can be any symbol you want, simply by typing its name after a colon (":"). In this practice, you can use it to indicate which model you want to train, for example `:ANN`, `:SVM`, `:DecisionTree` or `:kNN`.